In [ ]:
# Use in the google colab to connect the google cloud in order to get the dataset
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
# Use in google colab in order to get information about GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jun  6 06:40:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Use in google colab in order to get the root path
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
import sys
os.chdir('drive')

In [ ]:
import nltk
from functools import lru_cache
import re
import string
import numpy as np
import pandas as pd
import torch
from collections import Counter, defaultdict
from sklearn.preprocessing import OneHotEncoder
import torch.utils.data as Data
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import glob
from sklearn.utils import shuffle
import copy

In [ ]:
# view gpu version
if torch.cuda.is_available():
    device = torch.device("cuda")
    # print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(torch.cuda.device_count())
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1
Device name: Tesla V100-SXM2-16GB


In [ ]:
!/opt/bin/nvidia-smi

Sun Jun  6 06:40:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Copyright@cjhutto https://github.com/cjhutto/vaderSentiment 
class Modifiers:
    def __init__(self):
        C_INCR = 0.733
        B_INCR = 0.293
        B_DECR = -0.293

        N_SCALAR = -0.74
        self.NEGATE = \
            ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
             "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
             "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
             "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
             "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
             "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
             "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
             "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

        # booster/dampener 'intensifiers' or 'degree adverbs'
        # 

        self.BOOSTER_DICT = \
            {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR, "completely": B_INCR, "considerably": B_INCR,
             "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormously": B_INCR,
             "entirely": B_INCR, "especially": B_INCR, "exceptionally": B_INCR, "extremely": B_INCR,
             "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR,
             "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR,
             "fucking": B_INCR,
             "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR, "incredibly": B_INCR,
             "intensely": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
             "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
             "so": B_INCR, "substantially": B_INCR,
             "thoroughly": B_INCR, "totally": B_INCR, "tremendously": B_INCR,
             "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utterly": B_INCR,
             "very": B_INCR,
             "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
             "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
             "less": B_DECR, "little": B_DECR, "marginally": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
             "scarcely": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
             "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}


In [ ]:
class Preprocessor:
    def clean_text(self, text):
        # '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
        # make text lowercase
        text1 = text.lower()
        # remove square brackets
        text1 = re.sub('\[.*?\]', '', text1)
        #remove <>
        text1 = re.sub('<.*?>+', '', text1)
        text1 = re.sub('\(.*?\)', ' ', text1)
        text1 = re.sub('\{.*?\}', ' ', text1)
        # remove links
        text1 = re.sub('https?://\S+|www\.\S+', ' ', text1)
        # remove punctuation
        text1 = re.sub('[%s]' % re.escape(string.punctuation), '', text1)
        # remove \n
        # text = re.sub('\n', '', text)
        # remove numbers
        text = re.sub('\w*\d\w*', '', text)
        return text1

    def __init__(self):
        self.stem = lru_cache(maxsize=10000)(nltk.stem.SnowballStemmer('english').stem)
        # self.stopwords = stopwords.words('english')
        self.tokenize = nltk.tokenize.TreebankWordTokenizer().tokenize

    def __call__(self, text):
        text1 = self.clean_text(text)
        tokens = self.tokenize(text1)
        # tokens = [token for token in tokens if token not in self.stopwords]
        # tokens = [self.stem(token) for token in tokens]

        return tokens

In [ ]:

class DataLoader:
    def __init__(self, data, batch_size, k, context_window = 1, preprocessor=Preprocessor(), enc_tokens = OneHotEncoder(), enc_modifiers = OneHotEncoder(), modifier = Modifiers(), dev = 'cpu'):
        self.df = data # dataset
        self.dev = dev # choose "gpu" or "cpu"
        self.batch_size = batch_size # batch size
        self.context_window = context_window # context window size
        self.modifiers = modifier.BOOSTER_DICT.keys() # modifier lexicon
        self.enc_modifiers = enc_modifiers # one-hot encoder for modifiers
        # self.vocab_frequency = 5 # the lowest vocabulary frequency using in stance detection 
        self.padding_length = 75 # padding length for network
        self.fold_num = 10 # 10-fold cross validation
        self.negatives = modifier.NEGATE
        self.count = 0 # count neighbor modifier
        # self.first_sentence = 128 # Intercept the first paragraph using in stance detection
        # self.second_sentence = self.padding_length - self.first_sentence # Intercept the last paragraph using in stance detection

        self.apply_preprocessor(preprocessor) # clean text, tokenization
        enc_tokens.fit(self.vocab) # fit one-hot encoder for normal word lexicon
        self.enc_modifiers = enc_modifiers 
        self.enc_modifiers.fit(self.modifier_id_to_fit) # fit one-hot encoder for modifiers
        self.split(0.9, 0.1) # Divide the training and test dataset
        self.k_fold_partition(fold_num = self.fold_num) # Divide ten folds
        print("K Fold Partition Completed!")
        train, validation = self.k_fold_split(k)
        trainX, trainY = self.build_training_data(enc_tokens, train)
        validationX, validationY = self.build_training_data(enc_tokens, validation)
        testX, testY = self.build_training_data(enc_tokens, self.test)
        print(f'Train Dataset Shape : {trainX.shape}')
        print(f'Validation Dataset Shape : {validationX.shape}')
        print(f'Test Dataset Shape : {testX.shape}')
        self.train_dataset = self.get_batch(trainX, trainY)
        self.validation_dataset = self.get_batch(validationX, validationY)
        self.test_dataset = self.get_batch(testX, testY)



    def get_batch(self, X, Y):
        dataset = Data.TensorDataset(X, Y)
        loader = Data.DataLoader(
            dataset=dataset,
            batch_size=self.batch_size,
            shuffle=True,
            # num_workers=2,
        )
        return loader


    def remove_low_frequency(self, list):
        new_list = []
        for x in list:
            if x in self.token_to_count.keys():
                new_list.append(x)
        return new_list


    def apply_preprocessor(self, preprocessor):
        self.df['tokens'] = [preprocessor(s) for s in self.df['sentence']]
        self.df['tokens'] = [x[:self.padding_length]  if len(x) > self.padding_length else x for x in self.df['tokens']]
        # for index, row in self.df.iterrows():
        #     if len(row['tokens']) > self.max_length:
        #         row[toke]

        ######## Changed part for negatives ###########
        self.token_to_count = Counter([x for l in self.df['tokens'] for x in l if x not in self.modifiers and x not in self.negatives])
        self.modifiers_to_count = Counter([x for l in self.df['tokens'] for x in l if x in self.modifiers])
        self.negatives_to_count = Counter([x for l in self.df['tokens'] for x in l if x in self.negatives])
        ##############      End       #################

        # tmp_token_to_count = self.token_to_count.copy()
        # for index, value in tmp_token_to_count.items():
        #     if value <= self.vocab_frequency:
        #         self.token_to_count.pop(index)
        # self.df['tokens'] = [self.remove_low_frequency(x) for x in self.df['tokens']]
        self.max_length = self.get_max_length()
        print(f'Max Length : {self.max_length}')

        ######## Changed part for negatives ###########
        self.vocab = list([[term] for term in self.token_to_count.keys()])
        self.modifier_vocab = list([[term] for term in self.modifiers_to_count.keys()])
        self.negative_vocab = list([[term] for term in self.negatives_to_count.keys()])
        self.modifier_token_to_id = {self.modifier_vocab[i][0]: i + 1 for i in range(len(self.modifier_vocab))}
        self.modifier_id_to_fit = list([[term] for term in self.modifier_token_to_id.values()])
        ##############      End       #################

        print(f'Vocab Size : {len(self.vocab)}')
        print(f'Modifier Size : {len(self.modifier_vocab)}')
        print(f'Negative Size : {len(self.negative_vocab)}')
        # print(self.token_to_count)

    def get_max_length(self):
        max_length = 0
        for index, row in self.df.iterrows():
            ######## Changed part for negatives ###########
            token_list = [x for x in row['tokens'] if x not in self.modifiers and x not in self.negatives]
            ##############      End       #################
            tmp_length = len(token_list)
            if tmp_length > max_length:
                max_length = tmp_length
        return max_length

    def k_fold_partition(self, fold_num = 10):
        batch_size = int(len(self.train) / fold_num) # the number of data for each fold
        remain_num = len(self.train) - batch_size * fold_num # the remain data after partition
        self.fold_data = []
        fold_batch_list = [] # Average the remaining data to the folds
        for fold in range(fold_num):
          if remain_num > 0:
            remain_num -= 1
            fold_batch_list.append(batch_size + 1)
          else:
            fold_batch_list.append(batch_size)
        fold_index = 0 # The starting position of each division data
        for fold in range(fold_num):
          fold_texts = [fold_index, fold_index + fold_batch_list[fold]]
          self.fold_data.append(fold_texts)
          fold_index = fold_index + fold_batch_list[fold]


    def k_fold_split(self, k):
        print(f'K : {k}, fold_data[k] : {self.fold_data[k]}')
        validation = self.train.iloc[self.fold_data[k][0]: self.fold_data[k][1]]
        train = self.train.iloc[0: self.fold_data[k][0]].append(self.train.iloc[self.fold_data[k][1]:])
        train_num = len(self.train) - (self.fold_data[k][1] - self.fold_data[k][0])
        if train_num % self.batch_size == 0:
            self.no_batch = train_num / self.batch_size
        else:
            self.no_batch = int(train_num / self.batch_size) + 1
        print(f' index : {self.no_batch}')
        return train, validation



    def split(self, train, test):
        index = int(train * len(self.df))
        self.train = self.df.iloc[0:index]
        self.test = self.df.iloc[index:]

    def build_training_data(self, enc, df):
        X = []
        Y = []
        for index, row in df.iterrows():
            # build modifiers and negative words
            np_modifier = np.zeros(len(row['tokens']))
            np_negative = np.zeros(len(row['tokens'])) # innitial negative ids
            for index, value in enumerate(row['tokens']):
                if value in self.modifiers:
                    # work for modifiers
                    id = self.modifier_token_to_id[value]
                    np_modifier[index] = -1
                    np_negative[index] = -1 # mark the negative word to delete later
                    for i in range(1, self.context_window + 1):
                        if index - i > 0:
                          # ensure the affected words is none negative words or modifiers
                          # if the the affected words is negative words or modifiers, move once 
                          if row['tokens'][index - i] not in self.modifiers and row['tokens'][index - i] not in self.negatives:
                            np_modifier[index - i] = id
                          else:
                            if index - i - 1 > 0 and row['tokens'][index - i - 1] not in self.modifiers and row['tokens'][index - i - 1] not in self.negatives:
                              np_modifier[index - i - 1] = id
                        if index + i < len(row['tokens']):
                          # ensure the affected words is none negative words or modifiers
                          # if the the affected words is negative words or modifiers, move once 
                          if row['tokens'][index + i] not in self.modifiers and row['tokens'][index + i] not in self.negatives:
                            np_modifier[index + i] = id
                          else:
                            if index + i + 1 < len(row['tokens']) and row['tokens'][index + i + 1] not in self.modifiers and row['tokens'][index + i + 1] not in self.negatives:
                              np_modifier[index + i + 1] = id

                if value in self.negatives:
                    # work for negative words, similar with modifier method above
                    # The method uses np_negative[itself] = id - np_negative[itself] is to make
                    # sure that if there are two negative words, the switch will be twice
                    id = 1
                    np_negative[index] = -1
                    np_modifier[index] = -1 # mark the negative word to delete later
                    for i in range(1, self.context_window + 1):
                        if index - i > 0:
                          if row['tokens'][index - i] not in self.modifiers and row['tokens'][index - i] not in self.negatives:
                            np_negative[index - i] = id - np_negative[index - i]
                          else:
                            if index - i - 1 > 0 and row['tokens'][index - i - 1] not in self.modifiers and row['tokens'][index - i - 1] not in self.negatives:
                              np_negative[index - i - 1] = id - np_negative[index - i - 1]
                        if index + i < len(row['tokens']):
                          if row['tokens'][index + i] not in self.modifiers and row['tokens'][index + i] not in self.negatives:
                            np_negative[index + i] = id - np_negative[index + i]
                          else:
                            if index + i + 1 < len(row['tokens']) and row['tokens'][index + i + 1] not in self.modifiers and row['tokens'][index + i + 1] not in self.negatives:
                              np_negative[index + i + 1] = id - np_negative[index + i + 1]
                        
            # delete modifier and negative word
            deleted_index = [i for i in range(len(np_modifier)) if np_modifier[i] == -1]
            np_modifier = np.delete(np_modifier, deleted_index)
            np_negative = np.delete(np_negative, deleted_index)


            tmp = [enc.transform([[t]]).toarray()[0] for t in row['tokens'] if t not in self.modifiers and t not in self.negatives]
            for i in range(len(tmp)):
                # add the last as negative ids, the last before as modifier ids
                tmp[i] = np.append(tmp[i], np_modifier[i])
                tmp[i] = np.append(tmp[i], np_negative[i])
            if len(tmp) < self.max_length:
                pad_length = self.max_length - len(tmp)
                for i in range(pad_length):
                    tmp.append(np.append(np.zeros(len(self.vocab)), [-1, 0]))
            X.append(tmp)
            Y.append(row['label'])
        X = np.array(X)
        Y = np.array(Y)
        X = torch.from_numpy(X).float()
        Y = torch.from_numpy(Y).long()
        # print(f'Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : {X.shape}')
        # print(f'Input Label Shape : {Y.shape}')
        return X, Y


In [ ]:
class RelexNet(nn.Module):
    #Single step RNN.
    #input_size is char_vacab_size=26,hidden_size is number of hidden neurons，output_size is number of categories
    def __init__(self, vocabulary_size, modifier_size, len_seq, num_classes, enc):
        super(RelexNet, self).__init__()
        self.no_modifiers = np.zeros(modifier_size)
        self.num_classes = num_classes
        self.L = nn.Linear(vocabulary_size, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.M = nn.Linear(modifier_size, 1).cuda()
        # self.B = nn.Linear(len_seq, len_seq)
        self.enc = enc
        self.O = nn.Softmax(dim=1)

    def handle_modifier(self, modifier_ids, batch):
        # modifier_id = modifier_ids.cpu()
        modifier = []
        influence_flag = []
        add = 0.5 * Variable(torch.ones(batch, 1))
        for i in range(batch):
            if modifier_ids[i] == 0:
                modifier.append(self.no_modifiers)
                influence_flag.append(0)
                add[i] = 1
            elif modifier_ids[i] == -1:
                modifier.append(self.no_modifiers)
                influence_flag.append(0)
                add[i] = 0
            else:
                # print(modifier_ids[i])
                modifier.append(self.enc.transform([[modifier_ids[i].cpu()]]).toarray()[0])
                influence_flag.append(1)
        modifier = torch.tensor(modifier).float().cuda()
        influence_flag = torch.tensor(influence_flag).float()
        influence_flag = influence_flag.reshape(-1, 1).cuda()
        add = add.cuda()
        return modifier, influence_flag, add



    def forward(self, input, B_layer):
        # X.shape = (batch, seq_len, vocab_size)
        T = input.shape[1]
        batch = input.shape[0]
        predict_y = Variable(torch.zeros(batch, self.num_classes))

        if B_layer is None:
            B_layer = Variable(torch.zeros(batch, self.num_classes)).cuda()

        for t in range(T):
            tmp = input[:, t, :-2]
            modifier_ids = input[:, t, -2]
            negative_ids = input[:, t, -1]
            modifier, influence_flag, add = self.handle_modifier(modifier_ids, batch)

            L_onestep = self.L(tmp)
            L_onestep = self.dropout(L_onestep)
            L_onestep = torch.sigmoid(L_onestep)
            L_onestep = F.relu6(L_onestep)
            # modifier_score = torch.sigmoid(self.M(modifier))
            modifier_score = torch.relu(self.M(modifier))
            modifier_score = modifier_score * influence_flag
            modifier_score = modifier_score + add
            S_layer = L_onestep * modifier_score


            # This step is to switch the score of negative and positive
            # Using matrix operation is to guarantee the speed
            new_negative_ids = negative_ids.reshape(-1,1).repeat(1,2)
            ES_layer = S_layer[:,[1,0]]
            N_layer = ES_layer * new_negative_ids + S_layer * (1 - new_negative_ids)

            B_layer = torch.add(B_layer, N_layer)
            # print(B_layer)

            if self.num_classes == 1:
                O_layer[t] = F.sigmoid(B_layer)
            else:
                O_layer = self.O(B_layer)

        return O_layer, B_layer


In [ ]:
def train_model(dataset, model, optimizer, scheduler, num_epochs, dev):
    losses = []
    for epoch in range(num_epochs):
        # training mode
        # dataset.set_partition(dataset.train)
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        count = 0
        for x, y in dataset.train_dataset:
            # for every batch in the training dataset perform one update step of the optimizer.
            state = None
            model.zero_grad()
            y_h, state = model(x.to(dev), state)
            loss = F.cross_entropy(y_h, y.to(dev))
            optimizer.zero_grad()
            # scheduler.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()
            total_train_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
            count += 1
        average_train_loss = total_train_loss / count
        average_train_accuracy = total_train_correct / count
        print('{} optim: {}'.format(epoch + 1, optimizer.param_groups[0]['lr']))

        # validation mode
        model.eval()
        total_valid_loss = 0
        total_valid_correct = 0
        count = 0
        for x, y in dataset.validation_dataset:
            state = None
            y_h, state = model(x.to(dev), state)
            loss = F.cross_entropy(y_h, y.to(dev))
            total_valid_loss += loss.item()
            total_valid_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
            count += 1
        average_valid_loss = total_valid_loss / count
        losses.append((average_train_loss, average_valid_loss))
        average_valid_accuracy = total_valid_correct / count


        print(f'epoch {epoch + 1} accuracies: \t train: {average_train_accuracy}\t valid: {average_valid_accuracy} loss: {average_train_loss}\t')

    # test mode
    model.eval()
    total_test_correct = 0
    count = 0
    for x, y in dataset.test_dataset:
        state = None
        y_h, state = model(x.to(dev), state)
        total_test_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
        count += 1
    average_test_accuracy = total_test_correct / count

    print(f'test accuracy {average_test_accuracy}')

    return losses, (average_train_accuracy, average_valid_accuracy, average_test_accuracy)

In [ ]:
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1
k = 0

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
all_files = glob.glob("data/sentiment/*.csv")
data = pd.concat((pd.read_csv(f, header=None, index_col=None) for f in all_files))
data.columns = ['sentence', 'label']
data = data.sample(frac=1, random_state=1)
dataset = DataLoader(data, k=k, batch_size = batch_size, context_window=context_window, dev=dev)
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[8 * dataset.no_batch, 14 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
# torch.save(model, os.path.join('./relexnet_model_negative/classifier_M7.pth'))

cuda
Loading data...
Max Length : 68
Vocab Size : 5251
Modifier Size : 34
Negative Size : 24
K Fold Partition Completed!
K : 0, fold_data[k] : [0, 270]
 index : 76
Train Dataset Shape : torch.Size([2430, 68, 5253])
Validation Dataset Shape : torch.Size([270, 68, 5253])
Test Dataset Shape : torch.Size([300, 68, 5253])
Data Ready!
1 optim: 0.01
epoch 1 accuracies: 	 train: 0.6157620549201965	 valid: 0.6884921193122864 loss: 0.6809743478109962	
2 optim: 0.01
epoch 2 accuracies: 	 train: 0.7864583134651184	 valid: 0.8070436716079712 loss: 0.6401133105943078	
3 optim: 0.01
epoch 3 accuracies: 	 train: 0.8168859481811523	 valid: 0.788690447807312 loss: 0.6069461852312088	
4 optim: 0.01
epoch 4 accuracies: 	 train: 0.8427358269691467	 valid: 0.8080357313156128 loss: 0.5818977293215299	
5 optim: 0.01
epoch 5 accuracies: 	 train: 0.8622258901596069	 valid: 0.824404776096344 loss: 0.5618216191467486	
6 optim: 0.01
epoch 6 accuracies: 	 train: 0.8621710538864136	 valid: 0.8373015522956848 loss: 0

The code below is used for test on stance detection dataset before, 
which is not used at all

In [ ]:
# Abortion
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
# root_path = 'data/stance'
# files = ['abortion', 'gayRights', 'marijuana', 'obama']
# label = {'abortion' : 0, 'gayRights' : 1, 'marijuana' : 2, 'obama' : 3}
# list_sentence = []
# list_label = []
# for file in files:
#     path = os.path.join('data', 'stance', file, '*.data')
#     print(path)
#     all_files = glob.glob(path)
#     # print(all_files)
#     for label_file in all_files:
#         f = open(label_file, 'r', encoding='UTF-8')
#         tmp = f.read()
#         list_sentence.append(tmp)
#         list_label.append(label[file])

# dict = {'sentence': list_sentence, 'label': list_label}
# data = pd.DataFrame(dict)
# data = data.sample(frac=1, random_state=1)

data = pd.read_csv("abortion.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=256)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, enc_negative=dataset.enc_negatives, negative_size=len(dataset.negative_vocab),modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_abortion.pth'))

cuda
Loading data...
1915


TypeError: ignored

In [ ]:
# GayRights
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("gayRights.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, enc_negative=dataset.enc_negatives, negative_size=len(dataset.negative_vocab),modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_gayRights.pth'))

cuda
Loading data...
1376
Max Length : 183
Vocab Size : 1591
Modifier Size : 34
Negative Modifier Size : 28
 index : 18
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([550, 183, 1593])
Input Label Shape : torch.Size([550])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([412, 183, 1593])
Input Label Shape : torch.Size([412])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([414, 183, 1593])
Input Label Shape : torch.Size([414])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.5983796119689941	 valid: 0.6286407709121704	 valid loss: 0.6714368381546539	 precision: 0.38714499252615847	 recall: 0.6286407766990292	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.6435185074806213	 valid: 0.6286407709121704	 valid loss: 0.6698871221646522	 precision: 0.38714499252615847	 recall: 0.6286407766990292	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.6435185074806213	 valid: 0.6286407709121704	 valid loss: 0.6691

In [ ]:
# Marijuana
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("marijuana.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_marijuana.pth'))

cuda
Loading data...
626
Max Length : 180
Vocab Size : 851
Modifier Size : 32
 index : 8
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([250, 180, 852])
Input Label Shape : torch.Size([250])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([187, 180, 852])
Input Label Shape : torch.Size([187])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([189, 180, 852])
Input Label Shape : torch.Size([189])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.25901442766189575	 valid: 0.29411765933036804	 valid loss: 0.7172110469583521	 precision: 0.9649122807017544	 recall: 0.29411764705882354	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.4624398946762085	 valid: 0.6631016135215759	 valid loss: 0.6803992186638124	 precision: 0.41471571906354515	 recall: 0.6631016042780749	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.7454928159713745	 valid: 0.7058823704719543	 valid loss: 0.6602817796130869	 precision: 0.41640

In [ ]:
# Obama
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("obama.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_obama.pth'))

cuda
Loading data...
985
Max Length : 176
Vocab Size : 1299
Modifier Size : 31
 index : 13
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([394, 176, 1300])
Input Label Shape : torch.Size([394])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([295, 176, 1300])
Input Label Shape : torch.Size([295])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([296, 176, 1300])
Input Label Shape : torch.Size([296])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.523557722568512	 valid: 0.5559322237968445	 valid loss: 0.6914822905750598	 precision: 0.3685393258426966	 recall: 0.5559322033898305	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.5490384697914124	 valid: 0.5186440348625183	 valid loss: 0.6923747814307779	 precision: 0.7320574162679426	 recall: 0.5186440677966102	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.5918269157409668	 valid: 0.5186440348625183	 valid loss: 0.6914110822192693	 precision: 0.69230

In [ ]:
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
# root_path = 'data/stance'
# files = ['abortion', 'gayRights', 'marijuana', 'obama']
# label = {'abortion' : 0, 'gayRights' : 1, 'marijuana' : 2, 'obama' : 3}
# list_sentence = []
# list_label = []
# for file in files:
#     path = os.path.join('data', 'stance', file, '*.data')
#     print(path)
#     all_files = glob.glob(path)
#     # print(all_files)
#     for label_file in all_files:
#         f = open(label_file, 'r', encoding='UTF-8')
#         tmp = f.read()
#         list_sentence.append(tmp)
#         list_label.append(label[file])

# dict = {'sentence': list_sentence, 'label': list_label}
# data = pd.DataFrame(dict)
# data = data.sample(frac=1, random_state=1)

data = pd.read_csv("stance_dataset.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 4

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.005,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[3 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.9, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier2.pth'))

cuda
Loading data...
4902
Max Length : 189
Vocab Size : 3949
Modifier Size : 43
Negative Modifier Size : 29
 index : 62


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, os.path.join('classifier.pth'))

In [ ]:
!pip install kora
from kora import console
console.start()  # and click link